In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [7]:
import io
from google.colab import files
uploaded = files.upload()

Saving haraj_cars_arabic_ads.csv to haraj_cars_arabic_ads.csv


In [8]:
df = pd.read_csv(io.BytesIO(uploaded.get('haraj_cars_arabic_ads.csv')))

In [9]:
df

,ad_id,description,ad_cat
0,27773243,ورحمه الله جمس دينالي للبيع 2015 ماشي 44 ال...,SAL
1,27773329,عدد 4\nاصلي وكالة\nبدون ولا خدش\nكرت ماشاءالله...,PRT
2,27773360,سطحه هيدروليك نقل داخل الرياض وخارجها\n\n\n\n0...,SVC
3,27773216,... مطلوب شبك امامي الكبير الوسطي حق ماليبو 20...,REQ
4,27773228,جيب مديل 2011 جي ماشي 145 الف السياره من الوكا...,SAL
...,...,...,...
994,27777929,مجرد معلن التواصل عن طريق الجوال\n\n(اللكزس في...,SAL
995,27777916,إلسسلآم عليگم \nللبيع آنوآر آمآميه وخلفيه \nمن...,PRT
996,27777860,*وَلَا تَبْخَسُوا النَّاسَ أَشْيَاءَهُمْ وَلَا...,SAL
997,27777891,تويوتا لاندكروز 1997 للبيع \nسعودي GXR \nممشى...,SAL


In [10]:
df.columns=['id','text','class']

In [11]:
#df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      999 non-null    int64 
 1   text    999 non-null    object
 2   class   999 non-null    object
dtypes: int64(1), object(2)
memory usage: 23.5+ KB


processing

In [10]:
#df.describe()

In [12]:
def normalize(text):
    
    
    text = re.sub("[إأٱآا]", "ا", text)
    #text = re.sub("ة", "ة", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    text = re.sub("وما ", "ما ", text)
    text = re.sub("ما ", "ما", text)
    text = re.sub(" و ", "  ", text)
    
    
    
    
    #translator = str.maketrans('', '', punctuations_list)
    #text=text.translate(translator)
    
    
    text = re.sub('\W+'," ",text )
    text = re.sub(r'[?|؟|\$|/|.|=|!]',r' ',text) #remove specail char
    text = re.sub(r'[0-9]+', '', text) #remove numbers
    text = re.sub(r"\b[a-zA-Z]\b", "",text) #remove single letter 
    
    
    noise = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             =   |
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(noise, '', text)   #remove tanween
    text = re.sub('\s +', " ", text) #remove  more than one spaces
    
    
    return(text)

In [13]:
def Remove_duplicate_words(text):
    s1 =[]
    text = ' '.join(k for k,v in itertools.groupby(text.replace("&lt;/Sent&gt;","").split()))
    s = re.sub(r'\b(.+)(\s+\1\b)+', r'\1', text)
    return ''.join(s)

In [14]:
def Remove_duplicate_characters(text):
    result=[]
    TATWEEL = u"\u0640"
    text = text.replace(TATWEEL, '')
    text = ''.join(i for i, _ in itertools.groupby(text))
    result.append(text)
    return ''.join(result)

In [15]:
from nltk.tokenize import sent_tokenize,word_tokenize
import nltk
import csv
import string
import re
import itertools
df['text'] = df['text'].map(Remove_duplicate_characters)
df['text'] = df['text'].map(Remove_duplicate_words)
#df['text'] = df['text'].map(stopwordremove)
df['text'] = df['text'].map(normalize )

In [31]:
#view data
df

,id,text,class
0,27773243,ورحمه اله جمس دينالي لبيع ماشي الف وكسور الموا...,SAL
1,27773329,عد اصلي وكاله بدون ولا خدش كرت ماشاءاله السعر ...,PRT
2,27773360,سطحه هيدروليك نقل داخل الرياض وخارجها وسيله ال...,SVC
3,27773216,مطلوب شبك امامي الكبير الوسطي حق ماليبو الشكل...,REQ
4,27773228,جيب مديل جي ماشي الف السياره من الوكاله مع مال...,SAL
...,...,...,...
994,27777929,مجرد معلن التواصل عن طريق الجوال الكزس في مدين...,SAL
995,27777916,السلام عليكم لبيع انوار اماميه وخلفيه من موديل...,PRT
996,27777860,و ل ا ت ب خ س وا الن اس ا ش ي اء ه م و ل ا ت ...,SAL
997,27777891,تويوتا لاندكروز لبيع سعودي GXR مشي تقريبا بدي ...,SAL


SVM **CLASSIFIER**

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
#encoder =  LabelEncoder()

In [19]:
X= df['text']
y=df['class']

In [33]:
X

0      ورحمه اله جمس دينالي لبيع ماشي الف وكسور الموا...
1      عد اصلي وكاله بدون ولا خدش كرت ماشاءاله السعر ...
2      سطحه هيدروليك نقل داخل الرياض وخارجها وسيله ال...
3       مطلوب شبك امامي الكبير الوسطي حق ماليبو الشكل...
4      جيب مديل جي ماشي الف السياره من الوكاله مع مال...
                             ...                        
994    مجرد معلن التواصل عن طريق الجوال الكزس في مدين...
995    السلام عليكم لبيع انوار اماميه وخلفيه من موديل...
996     و ل ا ت ب خ س وا الن اس ا ش ي اء ه م و ل ا ت ...
997    تويوتا لاندكروز لبيع سعودي GXR مشي تقريبا بدي ...
998    بدل فورد فكتوريا المشي الف قابل لزياده ب جمس ا...
Name: text, Length: 999, dtype: object

In [21]:
y

0      SAL
1      PRT
2      SVC
3      REQ
4      SAL
      ... 
994    SAL
995    PRT
996    SAL
997    SAL
998    SAL
Name: class, Length: 999, dtype: object

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)

In [23]:
X_train.shape, y_train.shape

((699,), (699,))

In [24]:
from sklearn.svm import SVC

In [25]:
vector=SVC()

In [39]:
vectorizer2 = CountVectorizer()
vectorizer2.fit(X_train ,y_train )

CountVectorizer()

In [58]:
#print(vectorizer.vocabulary_)
#vector2 = vectorizer2.fit_transform(X_train)
vector2 = vectorizer2.fit_transform(['how are how you hpe you are fine'])

In [59]:
vector2

<1x5 sparse matrix of type '<class 'numpy.int64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [60]:
vectorizer2.get_feature_names

<bound method CountVectorizer.get_feature_names of CountVectorizer()>

In [61]:

print(vector2.shape)
print(type(vector2))
print(vector2.toarray())

(1, 5)
<class 'scipy.sparse.csr.csr_matrix'>
[[2 1 2 1 2]]


In [ ]:
#from sklearn.preprocessing import StandardScaler
#sc = StandardScaler()

In [ ]:
#X= encoder.fit_transform(df['text'])

In [ ]:
#y= encoder.fit_transform(df['class'])

In [ ]:
#model.fit(X_train , y_train)

In [ ]:
#le = preprocessing.encoder

In [ ]:
predictions = vector.predict(X_test) 

AttributeError: ignored

RNN 